# VacationPy

Author: Alex Schanne

This assignment will utilize skills in Python requests, APIs and JSON traversal in order to determine ideal vacation spots based on weather patterns change. More specifically this notebook will accomplish the following: 
<ul>
    <li> Create a heat map that displays the humidity for every city from the part I of the homework.
    <li> Narrow down the DataFrame to find your ideal weather condition. Based on criteria similar to the following:
       <ul>
        <li> A max temperature lower than 80 degrees but higher than 70.
        <li> Wind speed less than 10 mph.
        <li> Zero cloudiness.
        </ul>
    <li> Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
    <li> Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

#configuring gmaps
gmaps.configure(api_key = g_key)

In [2]:
#store weather_data.csv data into DataFrame
vacation_weather = pd.read_csv("weather_data.csv")

#fill NaN values and convert to float
vaca_weather = vacation_weather.dropna()
vaca_weather.head()

,city,country,lat,lng,cloudiness,humidity,max_temp,wind_speed
0,saint george,US,37.10,-113.58,1,13,102.20,13.87
1,palauig,PH,15.44,119.90,93,72,83.37,2.55
2,ushuaia,AR,-54.80,-68.30,75,80,33.80,21.92
3,provideniya,RU,64.38,-173.30,90,93,51.80,4.47
4,rikitea,PF,-23.12,-134.97,78,75,72.25,11.12


## Humidity Heatmapping

In [3]:
#store latitude and longitude as the weight and convert humidity to float
location = vaca_weather[['lat', 'lng']]
humid = vaca_weather['humidity'].astype(float)
print(humid.max())

100.0


In [4]:
#Plotting heatmap
#Defining map settings
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#Creating figure
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

#Creating heat layer
heat_layer = gmaps.heatmap_layer(location, weights=humid,
                                dissipating=False,max_intensity=100,
                                point_radius = 1.5)

#Adding heat layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

## Narrowing Down Vacation Location

In [12]:
#Using vacation location criteria to narrow down cities
ideal_temp = (vaca_weather.max_temp <= 95) & (vaca_weather.max_temp >= 70)
ideal_humid = vaca_weather.humidity < 25
ideal_wind = vaca_weather.wind_speed < 25
ideal_cloud = vaca_weather.cloudiness < 25
ideal_conditions = ideal_temp & ideal_humid & ideal_wind & ideal_cloud

#using boolean indexing to filter the dataframe
ideal_vaca = vaca_weather[ideal_conditions].dropna()
ideal_vaca.head()

,city,country,lat,lng,cloudiness,humidity,max_temp,wind_speed
97,bad bevensen,EG,24.09,32.91,24,15,91.15,7.94
347,reinosa,IR,30.81,56.56,0,22,71.22,3.71
389,brokopondo,US,44.37,-100.35,1,22,93.99,11.41


## Hotel Mapping

In [13]:
#Creating a DataFrame for the hotels
hotels = ideal_vaca
hotels['hotel'] = ''

hotels.head()

,city,country,lat,lng,cloudiness,humidity,max_temp,wind_speed,hotel
97,bad bevensen,EG,24.09,32.91,24,15,91.15,7.94,
347,reinosa,IR,30.81,56.56,0,22,71.22,3.71,
389,brokopondo,US,44.37,-100.35,1,22,93.99,11.41,


In [14]:
#params dictionary 
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

#using location data to recover hotel options
for index, row in hotels.iterrows():
    lat = row['lat']
    lng = row['lng']
    
    #change location each iteration 
    params['location'] = f'{lat},{lng}'
    
    #get the api base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #make request and convert to json
    response = requests.get(base_url, params=params).json()
    
    #update dataframe
    try:
        hotels.loc[index, "hotel"] = response['results'][0]['name']
    except:
        print("Could not find any results matching request.")
    continue

hotels.head()

,city,country,lat,lng,cloudiness,humidity,max_temp,wind_speed,hotel
97,bad bevensen,EG,24.09,32.91,24,15,91.15,7.94,Sofitel Legend Old Cataract Aswan
347,reinosa,IR,30.81,56.56,0,22,71.22,3.71,تالار قصر زرند(پردیس)
389,brokopondo,US,44.37,-100.35,1,22,93.99,11.41,Clubhouse Hotel & Suites


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
#</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels.iterrows()]
locations = hotels[["lat", "lng"]]

In [17]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations,
                                info_box_content = [info_box_template.format(**row) for index,
                                row in hotels.iterrows()])
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…